In [2]:
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
import backtesting
backtesting.set_bokeh_output(notebook=False)
import warnings
warnings.filterwarnings("ignore")
warnings.warn('Jupyter Notebook detected.')
import yfinance as yf
import datetime as dt
import talib



In [9]:
btc_data = yf.Ticker('BTC-USD')
btc = btc_data.history(start=dt.datetime(2025,6,1), end =dt.datetime(2025,7,1),interval= "5m" ).iloc[:, :4]#*10**-6
btc

,Open,High,Low,Close
Datetime,,,,
2025-06-01 00:00:00+00:00,104674.000000,104674.000000,104609.625000,104609.625000
2025-06-01 00:05:00+00:00,104598.093750,104602.023438,104590.867188,104590.867188
2025-06-01 00:10:00+00:00,104568.500000,104570.640625,104521.437500,104521.437500
2025-06-01 00:15:00+00:00,104525.187500,104525.187500,104478.304688,104478.304688
2025-06-01 00:20:00+00:00,104514.312500,104514.312500,104449.718750,104449.718750
...,...,...,...,...
2025-06-30 23:35:00+00:00,107158.265625,107158.265625,107134.867188,107134.867188
2025-06-30 23:40:00+00:00,107146.835938,107151.789062,107104.257812,107151.484375
2025-06-30 23:45:00+00:00,107158.312500,107164.351562,107124.992188,107124.992188


In [20]:
class Rsi0scillator(Strategy):
    lower_band = 30
    upper_band = 70
    rsi_window = 14
    def init(self):
        self.rsi = self.I(talib.RSI, self.data.Close, self.rsi_window)
        
    def next(self):
        if crossover(self.rsi, self.upper_band):
            self.position.close()
        elif crossover(self.rsi, self.lower_band):
            self.buy()
bt = Backtest(spy, Rsi0scillator, cash=10, commission=.02)
stat = bt.run()
bt.plot()
print(stat)


C:\Users\ludwi\AppData\Local\Temp\ipykernel_14468\992190952.py:13: UserWarning: Some prices are larger than initial cash value. Note that fractional trading is not supported. If you want to trade Bitcoin, increase initial cash, or trade μBTC or satoshis instead (GH-134).
  bt = Backtest(spy, Rsi0scillator, cash=10, commission=.02)
d:\Proyectos\SPP4backtesting\env\Lib\site-packages\backtesting\backtesting.py:954: UserWarning: time=299: Broker canceled the relative-sized order due to insufficient margin.
  warnings.warn(
d:\Proyectos\SPP4backtesting\env\Lib\site-packages\backtesting\backtesting.py:954: UserWarning: time=876: Broker canceled the relative-sized order due to insufficient margin.
  warnings.warn(
d:\Proyectos\SPP4backtesting\env\Lib\site-packages\backtesting\backtesting.py:954: UserWarning: time=1201: Broker canceled the relative-sized order due to insufficient margin.
  warnings.warn(
d:\Proyectos\SPP4backtesting\env\Lib\site-packages\backtesting\backtesting.py:954: UserWar

Start                     1993-01-29 00:00...
End                       2025-07-03 00:00...
Duration                  11842 days 23:00:00
Exposure Time [%]                         0.0
Equity Final [$]                         10.0
Equity Peak [$]                          10.0
Return [%]                                0.0
Buy & Hold Return [%]               2487.0045
Return (Ann.) [%]                         0.0
Volatility (Ann.) [%]                     0.0
CAGR [%]                                  0.0
Sharpe Ratio                              NaN
Sortino Ratio                             NaN
Calmar Ratio                              NaN
Alpha [%]                                 0.0
Beta                                      0.0
Max. Drawdown [%]                        -0.0
Avg. Drawdown [%]                         NaN
Max. Drawdown Duration                    NaN
Avg. Drawdown Duration                    NaN
# Trades                                    0
Win Rate [%]                      